In [1]:
# Teste I
from sklearn.datasets import make_blobs
import pandas as pd
import plotly.express as px
import numpy as np
from sklearn.cluster import KMeans
import time

# Teste II

from sklearn.neighbors import BallTree

In [2]:
def EuclideanDistance(a, b):
    return np.linalg.norm(a-b)

def BruteForceSearch(Principal, TestPoint):
    MinEuclideanDistance = np.inf
    for i in range(len(Principal)):
        Dist = EuclideanDistance(Principal.loc[i, ['x1', 'x2', 'x3']], TestPoint)
        if Dist < MinEuclideanDistance:
            ClosePoint = i
            MinEuclideanDistance = Dist
    return Principal['PointName'][ClosePoint]

# Inicia os parâmetros para a geração da base de dados

In [3]:
#Initial Parameters
NumberOfData = 10000
NumberOfCenters = 5
PlotData = False

# Geração da base de dados

In [4]:
X, y = make_blobs(n_samples = NumberOfData, centers = NumberOfCenters, n_features=3, cluster_std = 0.08, center_box = [0, 1])
DataFrame = pd.DataFrame(X, columns = ['x1', 'x2', 'x3'])
DataFrame['y'] = y
DataFrame['PointName'] = DataFrame.index
DataFrame['PointName'] = DataFrame['PointName'].apply(lambda x: 'p' + str(x))

In [5]:
DataFrame[['x1', 'x2', 'x3']].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
x1,10000.0,0.774987,0.209238,0.214030,0.574102,0.848735,0.947154,1.251290
x2,10000.0,0.586815,0.368230,-0.225411,0.286874,0.722756,0.900250,1.251639
x3,10000.0,0.434985,0.395192,-0.201891,0.105273,0.205281,0.863704,1.270432


# Plot da base de dados gerada

In [6]:
if PlotData:
    fig = px.scatter_3d(DataFrame, x="x1", y="x2", z="x3", color = 'y')
    fig.show()

# Remoção da variável de output dos dados gerados

In [7]:
DataFrame = DataFrame.drop(['y'], axis = 1)

# Declaração dos parâmetros do algoritmo K-Means

In [8]:
NClusters = 4

In [9]:
kmeans = KMeans(n_clusters=NClusters).fit(DataFrame[['x1', 'x2', 'x3']])
DataFrame['KMeans'] = kmeans.labels_

In [10]:
if PlotData:
    fig = px.scatter_3d(DataFrame, x="x1", y="x2", z="x3", color = 'KMeans')
    fig.show()

In [11]:
DataFrame

,x1,x2,x3,PointName,KMeans
0,0.982146,0.043250,0.046795,p0,0
1,0.509138,0.475933,1.168126,p1,3
2,0.836959,1.040822,0.903857,p2,1
3,1.004441,-0.081914,0.061374,p3,0
4,0.463283,0.511377,1.087871,p4,3
...,...,...,...,...,...
9995,0.443747,0.309338,0.994166,p9995,3
9996,0.911626,1.006318,0.172704,p9996,2
9997,0.956491,1.138530,0.133579,p9997,2
9998,0.949265,0.935713,0.336935,p9998,2


# Teste da metodologia de Cluster

- Etapas Globais:
    - Determina-se a quantidade de testes a ser realizado;
    - Determina-se os parâmetros de teste;
    - Gera-se uma lista de pontos de teste aleatórios.
    
- Busca exaustiva:
    - Para cada ponto aleatório na lista de pontos aleatórios faz-se a busca do ponto mais próximo.

- Busca no cluster:
    - Para cada ponto aleatório na lista de pontos aleatórios encontra a qual cluster ele pertence pelo algoritmo K-Means;
    - Faz-se a busca do ponto mais próximo olhando os pontos da base principal que fazem parte do cluster que o ponto aleatório foi determinado.


In [12]:
NpointsOfTest = 10 # Quantidade de pontos de teste

In [13]:
PointsOfTest = []
for i in range(NpointsOfTest):
    PointsOfTest.append(list(np.random.rand(1,3)[0]))
PointsOfTest = pd.DataFrame(PointsOfTest, columns = ['x1', 'x2', 'x3'])

In [14]:
MatchedPoint = []
TimeVector = []
for PointNumber in range(NpointsOfTest):
    StartTime = time.time()
    MatchedPoint.append(BruteForceSearch(DataFrame, PointsOfTest.loc[PointNumber,['x1', 'x2', 'x3']]))
    TimeVector.append(time.time() - StartTime)
    
TimeDataFrame = pd.DataFrame(TimeVector, columns = ['TempoBruteForce'])
PointFound = pd.DataFrame(MatchedPoint, columns = ['BruteForcePoint'])

In [15]:
MatchedPoint = []
TimeVector = []
for PointNumber in range(NpointsOfTest):
    StartTime = time.time()
    TestPoint = np.array(PointsOfTest.loc[PointNumber,['x1', 'x2', 'x3']]).reshape(1, -1)
    ClusterFound = kmeans.predict(TestPoint)[0]
    MatchedPoint.append(BruteForceSearch(DataFrame[DataFrame['KMeans'] == ClusterFound].reset_index(drop = True), TestPoint.reshape(-1)))
    TimeVector.append(time.time() - StartTime)
    
TimeDataFrame['TempoCluster'] = TimeVector
PointFound['ClusterPoint'] = MatchedPoint

# Teste da metodologia de BallTree

In [16]:
PointDict = pd.DataFrame.to_dict(DataFrame[['PointName']], orient = 'index')

In [17]:
tree = BallTree(DataFrame[['x1', 'x2', 'x3']], leaf_size=2)

In [18]:
MatchedPoint = []
TimeVector = []
for PointNumber in range(NpointsOfTest):
    StartTime = time.time()
    TestPoint = np.array(PointsOfTest.loc[PointNumber,['x1', 'x2', 'x3']]).reshape(1, -1)
    dist, ind = tree.query(TestPoint, k=1)
    MatchedPoint.append(PointDict[ind[0][0]]['PointName'])
    TimeVector.append(time.time() - StartTime)
    
TimeDataFrame['TempoBallTree'] = TimeVector
PointFound['BallTreePoint'] = MatchedPoint

# Avaliação dos resultados

In [19]:
TimeDataFrame.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
TempoBruteForce,10.0,7.638531,0.351041,7.333952,7.357250,7.476860,7.943671,8.225397
TempoCluster,10.0,1.901741,0.816521,1.352835,1.401884,1.411819,2.490628,3.367159
TempoBallTree,10.0,0.000900,0.000568,0.000000,0.000995,0.001000,0.001000,0.002003


In [20]:
PointFound

,BruteForcePoint,ClusterPoint,BallTreePoint
0,p9659,p9659,p9659
1,p3377,p3377,p3377
2,p5897,p5897,p5897
3,p3377,p3377,p3377
4,p5171,p5171,p5171
5,p6194,p6194,p6194
6,p8521,p8521,p8521
7,p1149,p1149,p1149
8,p8025,p8025,p8025
9,p7720,p7720,p7720
